In [16]:
import ollama
import joblib
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Load dataset
file_path = "opioid_notes_fully_unique.csv"
df = pd.read_csv(file_path)

# Assume "note" contains textual descriptions & "Opioid_Dependency_Label_x" is the target
df = df[["note", "Opioid_Dependency_Label_x"]].dropna()

# Convert text into numerical features using TF-IDF
vectorizer = TfidfVectorizer(max_features=500)  # Use top 500 features
X = vectorizer.fit_transform(df["note"])
y = df["Opioid_Dependency_Label_x"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model (Fixed: Removed probability=True)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save model & vectorizer
joblib.dump(model, "opioid_risk_model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")

# Function to predict opioid risk level
def predict_opioid_risk(doctor_notes):
    try:
        model = joblib.load("opioid_risk_model.pkl")
        vectorizer = joblib.load("vectorizer.pkl")

        # Transform input text into TF-IDF features
        input_features = vectorizer.transform([doctor_notes])

        # Predict risk probability
        probabilities = model.predict_proba(input_features)[0]

        # Ensure class order is correct (check y.unique())
        high_risk_prob = probabilities[1] if len(probabilities) > 1 else probabilities[0]

        # Define risk levels based on probability thresholds
        if high_risk_prob > 0.5:
            risk_level = "🔴 High Risk"
        else:
            risk_level = "🟢 Low Risk"

        return risk_level

    except Exception as e:
        return f"Error: {str(e)}"

# Ollama-based LLM analysis for generating recommendations
def ollama_opioid_advice(doctor_notes, risk_level):
    try:
        response = ollama.chat(
            model="llama3",  # Use a valid Ollama model
            messages=[
                {"role": "user", "content": f"Given the patient's condition: {doctor_notes}, and assessed risk level: {risk_level}, provide brief medical advice to reduce opioid dependency risk."}
            ]
        )
        return response["message"]["content"]
    except Exception as e:
        return f"Error: {str(e)}"

doctor_notes_input = "Patient has a history of chronic pain and opioid use with increased dependency over time."

risk_result = predict_opioid_risk(doctor_notes_input)
advice = ollama_opioid_advice(doctor_notes_input, risk_result)

print(f"Risk Level: {risk_result}\nRecommendations: {advice}")


Risk Level: 🔴 High Risk
Recommendations: Based on the patient's condition and high risk assessment, here are some brief medical advice to reduce opioid dependency risk:

1. **Gradual Taper**: Collaborate with the patient to develop a gradual taper plan to reduce their opioid dose over time. This can help minimize withdrawal symptoms and dependence.
2. **Alternative Pain Management**: Explore non-pharmacological and pharmacological alternatives for pain management, such as:
	* Physical therapy and exercise
	* Cognitive-behavioral therapy (CBT)
	* Acupuncture
	* Massage therapy
	* Gabapentin or pregabalin (if not contraindicated)
3. **Non-Opioid Analgesics**: Consider prescribing non-opioid analgesics, such as:
	* NSAIDs (e.g., acetaminophen, ibuprofen) for mild to moderate pain
	* Tramadol or tapentadol for moderate to severe pain (if necessary)
4. **Behavioral Interventions**: Encourage the patient to adopt healthy habits, such as:
	* Regular exercise and physical activity
	* Stress ma

In [14]:
doctor_notes_input = "Patient complains of persistent back pain but is hesitant to take strong medications. She denies any history of substance abuse. Requests alternative pain relief options."

# 🔹 Predict risk dynamically
risk_result = predict_opioid_risk(doctor_notes_input)
advice = ollama_opioid_advice(doctor_notes_input, risk_result)

# 🔹 Output result
print(f"{risk_result} of Opioid Addiction\nRecommendations: {advice}")

🟢 Low Risk of Opioid Addiction
Recommendations: Based on the patient's condition and assessment, here are some brief medical advice to reduce opioid dependency risk:

**Non-Opioid Options**

1. **Physical Therapy**: Encourage gentle exercises and stretches to strengthen back muscles, improve posture, and reduce pain.
2. **Heat or Cold Therapy**: Recommend applying heat packs or cold compresses to the affected area to relieve tension and discomfort.
3. **Acupuncture**: Consider referring the patient to a licensed acupuncturist for non-pharmacological pain relief.
4. **Massage Therapy**: Suggest gentle massage techniques, such as trigger point therapy or myofascial release, to relax muscles and reduce pain.

**Opioid-Sparing Medications**

1. **Topical Creams**: Prescribe topical creams containing capsaicin or lidocaine for local pain relief.
2. **Muscle Relaxants**: Offer muscle relaxants like cyclobenzaprine (Flexeril) or metaxalone (Skelaxin) as alternatives to opioids.
3. **Anticonvu